# Introduction to AgenTools

## Why should I exist?
AgenTools aims to be what PyTorch Lightning is to PyTorch, but for LLM-based **assistants**, **copilots** and **agents**. Similar to how the training loop in deep learning essentially stays the same, AgenTools handles the boilerplate code and give you a both high-level and highly customizable interface.

On the other hand, AgenTools is explicitly **not trying to be another LangChain, LlamaIndex or Haystack**. We want to abstract only the boilerplate away, but data connections, long pipelines, many API connections, etc. are not the focus of AgenTools, in order to be as lightweight as possible.


## What can I do?
There are, broadly speaking, two types of customization you might want:
- **Assistant Implementation:** How each step of the "assistant loop" is implemented, and
- **Assistant Integration:** How your application then reacts to what the assistant does.

Often enough, the provided **assistant implementations** will be sufficient, and building an app around the assistant will be the main task. Therefore, let's first look at the latter:

### Assistant Integration

To allow maximum efficiency and flexibility, AgenTools centers around the concept of asynchronous **event generators**, which `yield` events that you can freely handle in your application:

In [1]:
from datetime import datetime
from agentools import *


@function_tool
async def get_date_time():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


ass = Assistant(tools=get_date_time)

async for event in ass.response_events("Hey, what time is it?"):
    match event:
        # ========== Normal Events ========== #
        case ResponseStartEvent():
            print(f"{event.prompt} -> {event.model}")

        case TextMessageEvent():
            print(f"[Assistant]: {event.content}")

        case ToolCallsEvent():
            for call in event.tool_calls:
                f = call.function
                print(f"[Assistant]: Calling {f.name}({f.arguments})...")

        case ToolResultEvent():
            print(f"[{event.tool_call.function.name}(...)]: {event.result}")

        # ========== Error Events ========== #
        case MaxTokensExceededEvent():
            print("[ERROR] Max token reached.")

Hey, what time is it? -> gpt-3.5-turbo
[Assistant]: Calling get_date_time({})...
[get_date_time(...)]: 2024-06-03 18:11:12
[Assistant]: The current time is 18:11:12 on June 3, 2024.


Replace the above `print`s with a frontend connection, and you have a simple chatbot. Use `yield`s to turn it into a generator inside a FastAPI endpoint, and your endpoint supports streaming response to the client. `match` any events that you need to listen to.

This is as concise and flexible as it gets!

### Assistant Implementation

While a simple LLM is as simple as *Prompt In -> Completion Out*, assistants with tools, and streaming support is much more complex to implement.

The following pseudocode shows the loop that every assistant follows, for a single prompt with tools support:

```python
def assistant_loop(history, prompt, tools):
    # add the prompt to the conversation history
    history.append(user=prompt)

    while True:
        # generate the completion
        completion = generate_completion(messages)

        # select the response message, if there are multiple choices
        response = completion.choices[0]
        history.append(assistant=message)

        # do something with the assistant's response
        handle_text_content(response.content)

        # (zero, one or more) tools called by the assistant
        for tool_call in response.tool_calls:
            tool_output = call_tool(tool_call, tools)
            history.append(tool=tool_output)
        
        # if no tool was called, the response is final
        if not response.tool_calls:
            break
```

This loop gets much bigger if streaming must also be handled. Here's what `generate_completion` from above could look like:

```python
def generate_completion(messages):
    # call the underlying LLM, e.g. through an API
    completion_stream = call_llm_api(messages)

    partial_completion = ... # gather the tokens here
    
    # the stream yields new tokens as they are generated by the LLM
    for delta_chunk in completion_stream:
        # accumulate the new chunks of tokens
        partial_completion.extend(delta_chunk)
        
        # select the response message, if there are multiple choices
        partial_response = partial_completion.choices[0]

        # do something with the assistant's partial response
        handle_partial_text_content(partial_response.content)

        # do something with the assistant's partial tool calls
        handle_partial_tool_calls(partial_response.tool_calls)
    
    # all tokens have been generated and gathered
    return partial_completion
```

And again, the `handle_partial_tool_calls` function could be quite complex, if you want to do something fancy with them in the frontend, such as displaying a preview of the tool's output from the partial completion.  
AgenTools also offers additional utilities to ease handling partial tool calls, e.g. a custom JSON parser that can deal with incomplete JSON objects, or preview functions that you can define for each tool.